In [56]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from PIL import Image
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


AUTOTUNE=tf.data.experimental.AUTOTUNE

tf.__version__

'2.1.0'

In [95]:
DATA_PATH = r'C:\Users\subrahm\Documents\breast-cancer-detection\Breast-Cancer-Detection\data\split_data'

In [97]:
num_images = len(list(glob.glob(os.path.join(img_dir, '*/*/*.png'))))
"total images: {}".format(num_images)

'total images: 4627'

In [98]:
train_dir = os.path.join(DATA_PATH,'train')
val_dir = os.path.join(DATA_PATH,'val')

In [ ]:
Image.open(list(glob.glob(train_dir+'/*/*.png'))[0])

In [99]:
CLASSES = np.array([item for item in os.listdir(train_dir)])
"classes: {}".format(CLASSES)

"classes: ['normal_tissue' 'tumor']"

In [104]:
print('Train:')
for cls in CLASSES:
    print(f"number of images belonging to {cls} class: {len(os.listdir(os.path.join(train_dir,cls)))}")
    
print('\nValidation:')
for cls in CLASSES:
    print(f"number of images belonging to {cls} class: {len(os.listdir(os.path.join(val_dir,cls)))}")

Train:
number of images belonging to normal_tissue class: 1864
number of images belonging to tumor class: 1836

Validation:
number of images belonging to normal_tissue class: 233
number of images belonging to tumor class: 229


In [105]:
BATCH_SIZE = 128
EXPOCHS = 15
IMG_HEIGHT = 256
IMG_WIDTH = 256

In [106]:
train_list_ds = tf.data.Dataset.list_files(os.path.join(train_dir, '*/*.png'))
val_list_ds = tf.data.Dataset.list_files(os.path.join(val_dir, '*/*.png'))

In [107]:
for f in train_list_ds.take(1):
    print(f)
    
for f in val_list_ds.take(1):
    print(f)

tf.Tensor(b'C:\\Users\\subrahm\\Documents\\breast-cancer-detection\\Breast-Cancer-Detection\\data\\split_data\\train\\tumor\\A_1434_1.RIGHT_CC.ROI.01.png', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\subrahm\\Documents\\breast-cancer-detection\\Breast-Cancer-Detection\\data\\split_data\\val\\normal_tissue\\A_0112_1.LEFT_CC.ROI.04.png', shape=(), dtype=string)


In [108]:
def get_label(file_path):
    parts = tf.strings.split(file_path, '\\')
    return parts[-2] == classes

In [109]:
def decode_image(img):
    img = tf.image.decode_png(img, channels=3) 
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 

In [110]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_image(img)
    return img, label

In [111]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [112]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
    
for image, label in val_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())    

Image shape:  (256, 256, 3)
Label:  [False  True]
Image shape:  (256, 256, 3)
Label:  [ True False]


In [117]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)

(256, 256, 3)
(2,)
